# Modelling with mlflow

__Goal__: Add `mlflow` to a simpliefied version of notebook `Modelling.ipynb`. 

### Import

In [1]:
%load_ext autoreload
%autoreload 2
import joblib
import mlflow
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
from pathlib import Path
from pprint import pprint

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier

from weather.transformers.skl_transformer_makers import (
    FeatureNames,
    TargetChoice,
    make_dataset_ingestion_transformer,
    make_target_creation_transformer,
    make_remove_horizonless_rows_transformer, 
    make_predictors_feature_engineering_transformer,
)
from weather.data.prep_datasets import (
    prepare_binary_classification_tabular_data, 
    transform_dataset_and_create_target,
)
from weather.mlflow.tracking import (
    get_best_run,
    explore_best_runs, 
    load_model_from_run,
)
from weather.mlflow.registry import (
    register_model_from_run, 
    get_latest_model_versions,
    load_production_model,
    transition_model_to_production,
)
from weather.helpers.utils import camel_to_snake
from weather.models.skl_tracked_train_models import (
    Experiment,
    train_and_evaluate_with_tracking,
)
from weather.models.skl_train_models import (
    train_and_evaluate, 
    score_evaluation,
)

### Set the directory paths

In [2]:
data_dir =  Path.cwd().parent / "data"
models_dir = Path.cwd().parent / "models"
models_dir.mkdir(exist_ok=True)

### Set the transformers parameters

In [3]:
# Set column names
oldnames_newnames_dict = {
    "Temperature_C": "Temperature", 
    "Apparent_Temperature_C": "Apparent_temperature",
    "Wind_speed_kmph": "Wind_speed",
    "Wind_bearing_degrees": "Wind_bearing",
    "Visibility_km": "Visibility",
    "Pressure_millibars": "Pressure",
    "Weather_conditions": "Weather"}

# Select "predicted", aka "Weather" in 4 hours
target_name = "Weather"
horizon = 4
target_choice = TargetChoice(target_name, horizon)

# Select feature_names
feature_names = FeatureNames(
    numerical=[
        "Temperature",
        "Humidity",
        "Wind_speed",
        "Wind_bearing",
        "Visibility",
        "Pressure",
    ],
    categorical=[],  # Add or remove "Weather", "Month" to the predictors
)

### Build the transformers

In [4]:
dataset_ingestion_transformer = make_dataset_ingestion_transformer(target_choice, oldnames_newnames_dict)
remove_horizonless_rows_transformer = make_remove_horizonless_rows_transformer(target_choice)
target_creation_transformer = make_target_creation_transformer(target_choice)                       
predictors_feature_engineering_transformer = make_predictors_feature_engineering_transformer(feature_names, target_choice)

### Read the development raw data

In [5]:
df = pd.read_csv(data_dir / 'weather_dataset_raw_development.csv')
df.head(1)

,S_No,Timestamp,Location,Temperature_C,Apparent_Temperature_C,Humidity,Wind_speed_kmph,Wind_bearing_degrees,Visibility_km,Pressure_millibars,Weather_conditions
0,2881,2006-01-01 00:00:00+00:00,"Port of Turku, Finland",1.161111,-3.238889,0.85,16.6152,139,9.9015,1016.15,rain


### Transform the raw dataset and split it

In [6]:
# Three transformers: "dataset__ingestion_transformer", "remove_horizonless_rows_transformer", "target_creation_transformer"
transformed_data, created_target = transform_dataset_and_create_target(
    df,   
    dataset_ingestion_transformer,
    remove_horizonless_rows_transformer,
    target_creation_transformer,
)

# Split the dataset
dataset = prepare_binary_classification_tabular_data(
    transformed_data,
    created_target,
)

### Define a set of candidate models

In [7]:
random_state = 1234
max_depth = 4
max_iter = 10_000

models = {
    "DecisionTree": {
        "model": DecisionTreeClassifier(max_depth=max_depth, random_state=random_state),
    },
    "LinearSvc": {
        "model": LinearSVC(max_iter=max_iter, random_state=random_state),
    },
    "LogisticRegression": {
        "model": LogisticRegression(),
    },
    "RandomForest": {
        "model": RandomForestClassifier(max_depth=max_depth, random_state=random_state),
    },
}

### Model

### Model with mlflow

WARNING: The method `.fit()` might already been called by `predictors_feature_engineering_transformer` in section `Model` above, within `train_and_evaluate()`. Even in that case, it would not affect the results below (`.fit()` would be called twice).

In [8]:
MLFlow_URI = 'http://127.0.0.1:5000'
experiment_name = "tune_random_forest_with_full_pipeline"

experiment = Experiment(MLFlow_URI, experiment_name)

##### Remove previous runs

In [9]:
# mlflow.set_tracking_uri(MLFlow_URI)
# runs = mlflow.search_runs(experiment_names=[experiment_name)
# for run_id in runs["run_id"]:
#     mlflow.delete_run(run_id)
#     print(f"Deleted run {run_id}")

In [10]:
# classifiers_list = [LogisticRegression, LinearSVC, DecisionTreeClassifier, RandomForestClassifier]
classifiers_list = [RandomForestClassifier]
train_and_evaluate_with_tracking(dataset, predictors_feature_engineering_transformer, classifiers_list, f1_score, experiment)

### Model Tracking

In [11]:
# Retrieve a list of runs from the experiment
runs = explore_best_runs(experiment, to_dataframe=False) # mlflow.store.entities.paged_list.PagedList of length 2

In [12]:
# Fetch the best run from the specified experiment, 'test_of_clustering_plus_classification'
best_run = get_best_run(experiment) # mlflow.entities.run.Run

In [13]:
# Load model artifacts from the best_run within the specified experiment, 'test_of_clustering_plus_classification'
loaded_artifacts = load_model_from_run(experiment.tracking_server_uri, best_run) # mlflow.pyfunc.PyFuncModel

In [14]:
# Extract the data transformer and classifier objects from the loaded model artifacts. Provide training data as input to predict() for schema verification.
transformer, classifier = loaded_artifacts.predict(dataset.train_x) # weather.transformers.skl_transformer_utilities.SimpleCustomPipeline, sklearn.ensemble._forest.RandomForestClassifier

In [15]:
transformer

SimpleCustomPipeline(steps=[('add_columns_year_month_day_hour',
                             AddColumnsYearMonthDayHourFromIndexTransformer()),
                            ('weather',
                             WeatherTransformer(feature_name='Weather')),
                            ('fill_initial_rows_nans',
                             FillInitialRowsWithBfillTransformer(number_of_rows=1)),
                            ('nans_imputation', NaNsImputationTransformer()),
                            ('outliers',
                             Pipeline(steps=[('inpute_humidity_outlier...
                                              ImputeOutliersTransformer(column_name='Pressure'))])),
                            ('one_hot_encoder_and_standard_scaler',
                             DataFrameColumnTransformer(transformers=[('cat',
                                                                       SimpleCustomPipeline(steps=[('onehot',
                                                                                                    OneHotEncoderDataFrame())]),
                                                                       []),
                                                                      ('num',
                                                                       SimpleCustomPipeline(steps=[('scaler',
                                                                                                    TransformerToDataFrame(base_transformer=StandardScaler()))]),
                                                                       ['Temperature',
                                                                        'Humidity',
                                                                        'Wind_speed',
                                                                        'Wind_bearing',
                                                                        'Visibility',
                                                                        'Pressure'])]))])

In [16]:
classifier

RandomForestClassifier()

In [17]:
# Evaluate the accuracy of the pair (data transformer, classifier) on the dataset
score_evaluation(f1_score, transformer, classifier, dataset)

Score(score_name='f1_score', train=1.0, val=0.955, test=0.952)

### Model Registry

In [18]:
# Retrieve the best run from the 'experiment' using default metric 'valid_accuracy'
best_run = get_best_run(experiment)

In [19]:
# Register the best run's model as 'random_forest'
register_model_from_run(experiment.tracking_server_uri, best_run, 'random_forest')

Registered model 'random_forest' already exists. Creating a new version of this model...
2024/02/11 17:00:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest, version 2
Created version '2' of model 'random_forest'.


In [20]:
# Retrieve the latest versions of the 'random_forest' model to observe the function behaviors
get_latest_model_versions(experiment.tracking_server_uri, 'random_forest') 

[{'version': '1', 'stage': 'Production'}, {'version': '2', 'stage': 'None'}]

In [21]:
# Transition model version '1' of 'random_forest' to the 'Production' stage 
transition_model_to_production(experiment.tracking_server_uri, 'random_forest', '1') 

In [22]:
# Retrieve the latest versions of the 'random_forest' model to observe function behavior
get_latest_model_versions(experiment.tracking_server_uri, 'random_forest') 

[{'version': '1', 'stage': 'Production'}, {'version': '2', 'stage': 'None'}]

In [23]:
# Load the 'random_forest' model artifacts deployed in the 'Production' stage
loaded_artifacts = load_production_model(experiment.tracking_server_uri, 'random_forest')

In [24]:
# Extract the data transformer and classifier objects from the loaded model artifacts. Provide training data as input to predict() for schema verification.
transformer, classifier = loaded_artifacts.predict(dataset.train_x)

In [25]:
# Evaluate the accuracy of the pair (data transformer, classifier) on the dataset 
score_evaluation(f1_score, transformer, classifier, dataset)

Score(score_name='f1_score', train=1.0, val=0.955, test=0.952)